# LSTM Models for Sarcasm Detection

This notebook implements LSTM models using both PyTorch and TensorFlow to detect sarcasm in news headlines using the DeteksiSarkasme.json dataset.

## Requirements:
1. Build deep learning models (PyTorch and TensorFlow)
2. Use evaluation metrics (Accuracy, Precision, Recall, F1-Score, AUC, ROC)
3. Visualize accuracy matrix and loss
4. Hyperparameter tuning with Keras Tuner (TensorFlow)
5. Achieve minimum 80% accuracy on training and testing sets

## Dataset
The dataset contains news headlines with binary labels:
- 0: Not sarcastic
- 1: Sarcastic

In [ ]:
!pip install torch torchvision torchaudio
!pip install tensorflow
!pip install keras-tuner
!pip install scikit-learn
!pip install matplotlib seaborn
!pip install wordcloud
!pip install nltk

In [ ]:
# Import necessary libraries
import json
import numpy as np
import pandas as pd
import re
import string
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# Machine Learning libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, classification_report

# PyTorch libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

# TensorFlow libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# Keras Tuner for hyperparameter tuning
import keras_tuner as kt

# NLTK for text preprocessing
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
tf.random.set_seed(42)

print("All libraries imported successfully!")

## Data Loading and Exploration

In [ ]:
# Load the dataset
data_path = '../Dataset/DeteksiSarkasme.json'

# Read JSON data
data = []
with open(data_path, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

# Convert to DataFrame
df = pd.DataFrame(data)
print(f"Dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print("\nFirst few rows:")
print(df.head())

# Basic statistics
print(f"\nDataset Info:")
print(f"Total samples: {len(df)}")
print(f"Sarcastic samples: {df['is_sarcastic'].sum()}")
print(f"Non-sarcastic samples: {len(df) - df['is_sarcastic'].sum()}")
print(f"Sarcasm ratio: {df['is_sarcastic'].mean():.2%}")

# Check for missing values
print(f"\nMissing values:")
print(df.isnull().sum())

In [ ]:
# Visualize class distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
df['is_sarcastic'].value_counts().plot(kind='bar')
plt.title('Class Distribution')
plt.xlabel('Is Sarcastic')
plt.ylabel('Count')
plt.xticks([0, 1], ['Non-Sarcastic', 'Sarcastic'], rotation=0)

plt.subplot(1, 2, 2)
df['is_sarcastic'].value_counts().plot(kind='pie', autopct='%1.1f%%')
plt.title('Class Distribution (Percentage)')
plt.ylabel('')

plt.tight_layout()
plt.show()

# Text length analysis
df['headline_length'] = df['headline'].str.len()
df['word_count'] = df['headline'].str.split().str.len()

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.hist(df['headline_length'], bins=50, alpha=0.7, edgecolor='black')
plt.title('Distribution of Headline Length (Characters)')
plt.xlabel('Number of Characters')
plt.ylabel('Frequency')

plt.subplot(1, 3, 2)
plt.hist(df['word_count'], bins=30, alpha=0.7, edgecolor='black')
plt.title('Distribution of Word Count')
plt.xlabel('Number of Words')
plt.ylabel('Frequency')

plt.subplot(1, 3, 3)
sarcastic_lengths = df[df['is_sarcastic'] == 1]['word_count']
non_sarcastic_lengths = df[df['is_sarcastic'] == 0]['word_count']
plt.hist(sarcastic_lengths, bins=30, alpha=0.7, label='Sarcastic', edgecolor='black')
plt.hist(non_sarcastic_lengths, bins=30, alpha=0.7, label='Non-Sarcastic', edgecolor='black')
plt.title('Word Count by Class')
plt.xlabel('Number of Words')
plt.ylabel('Frequency')
plt.legend()

plt.tight_layout()
plt.show()

print(f"Average headline length: {df['headline_length'].mean():.2f} characters")
print(f"Average word count: {df['word_count'].mean():.2f} words")
print(f"Max word count: {df['word_count'].max()} words")
print(f"Min word count: {df['word_count'].min()} words")

In [ ]:
# Word Cloud Visualization
plt.figure(figsize=(20, 10))

# Sarcastic headlines
sarcastic_text = ' '.join(df[df['is_sarcastic'] == 1]['headline'])
plt.subplot(1, 2, 1)
wordcloud_sarcastic = WordCloud(width=800, height=400, background_color='white').generate(sarcastic_text)
plt.imshow(wordcloud_sarcastic, interpolation='bilinear')
plt.title('Word Cloud - Sarcastic Headlines', fontsize=16)
plt.axis('off')

# Non-sarcastic headlines
non_sarcastic_text = ' '.join(df[df['is_sarcastic'] == 0]['headline'])
plt.subplot(1, 2, 2)
wordcloud_non_sarcastic = WordCloud(width=800, height=400, background_color='white').generate(non_sarcastic_text)
plt.imshow(wordcloud_non_sarcastic, interpolation='bilinear')
plt.title('Word Cloud - Non-Sarcastic Headlines', fontsize=16)
plt.axis('off')

plt.tight_layout()
plt.show()

## Text Preprocessing

In [ ]:
# Text preprocessing functions
def clean_text(text):
    """Clean and preprocess text data"""
    # Convert to lowercase
    text = text.lower()
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)
    
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove extra whitespace
    text = ' '.join(text.split())
    
    return text

# Apply text preprocessing
df['clean_headline'] = df['headline'].apply(clean_text)

# Remove stop words (optional - can be beneficial for some models)
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    """Remove stopwords from text"""
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word not in stop_words and len(word) > 2]
    return ' '.join(filtered_tokens)

# Create version with stopwords removed
df['clean_headline_no_stop'] = df['clean_headline'].apply(remove_stopwords)

# Display examples
print("Original vs Cleaned Headlines:")
print("-" * 50)
for i in range(5):
    print(f"Original: {df.iloc[i]['headline']}")
    print(f"Cleaned: {df.iloc[i]['clean_headline']}")
    print(f"No Stopwords: {df.iloc[i]['clean_headline_no_stop']}")
    print("-" * 50)

## PyTorch LSTM Model

In [ ]:
# PyTorch Dataset Class
class SarcasmDataset(Dataset):
    def __init__(self, texts, labels, vocab_to_int, max_len=100):
        self.texts = texts
        self.labels = labels
        self.vocab_to_int = vocab_to_int
        self.max_len = max_len
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        # Convert text to integers
        text_ints = [self.vocab_to_int.get(word, 0) for word in text.split()]
        
        # Pad or truncate to max_len
        if len(text_ints) > self.max_len:
            text_ints = text_ints[:self.max_len]
        else:
            text_ints = text_ints + [0] * (self.max_len - len(text_ints))
            
        return torch.tensor(text_ints, dtype=torch.long), torch.tensor(label, dtype=torch.float32)

# Create vocabulary
def create_vocab(texts, min_freq=2):
    """Create vocabulary from texts"""
    # Count word frequencies
    word_counts = Counter()
    for text in texts:
        words = text.split()
        word_counts.update(words)
    
    # Create vocabulary (words with frequency >= min_freq)
    vocab = ['<PAD>', '<UNK>'] + [word for word, count in word_counts.items() if count >= min_freq]
    vocab_to_int = {word: idx for idx, word in enumerate(vocab)}
    int_to_vocab = {idx: word for word, idx in vocab_to_int.items()}
    
    return vocab_to_int, int_to_vocab

# Prepare PyTorch data
texts = df['clean_headline'].tolist()
labels = df['is_sarcastic'].tolist()

# Create vocabulary
vocab_to_int, int_to_vocab = create_vocab(texts)
vocab_size = len(vocab_to_int)

print(f"Vocabulary size: {vocab_size}")
print(f"Sample vocabulary: {list(vocab_to_int.items())[:10]}")

# Split data
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42, stratify=labels)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
print(f"Test set size: {len(X_test)}")

# Create datasets
max_len = 50  # Maximum sequence length
train_dataset = SarcasmDataset(X_train, y_train, vocab_to_int, max_len)
val_dataset = SarcasmDataset(X_val, y_val, vocab_to_int, max_len)
test_dataset = SarcasmDataset(X_test, y_test, vocab_to_int, max_len)

# Create data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# PyTorch LSTM Model
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, dropout=0.3):
        super(LSTMClassifier, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, 
                           batch_first=True, dropout=dropout, bidirectional=True)
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(hidden_dim * 2, 64)  # *2 for bidirectional
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        # Embedding
        embedded = self.embedding(x)
        
        # LSTM
        lstm_out, (hidden, _) = self.lstm(embedded)
        
        # Use the last output
        output = lstm_out[:, -1, :]
        
        # Fully connected layers
        output = self.dropout(output)
        output = torch.relu(self.fc1(output))
        output = self.dropout(output)
        output = self.fc2(output)
        output = self.sigmoid(output)
        
        return output.squeeze()

# Model parameters
embedding_dim = 100
hidden_dim = 128
num_layers = 2
dropout = 0.3

# Initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

pytorch_model = LSTMClassifier(vocab_size, embedding_dim, hidden_dim, num_layers, dropout)
pytorch_model.to(device)

# Loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(pytorch_model.parameters(), lr=0.001)

print(f"Model architecture:")
print(pytorch_model)

In [ ]:
# Training function for PyTorch
def train_pytorch_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20):
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0
        
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            predicted = (output > 0.5).float()
            train_total += target.size(0)
            train_correct += (predicted == target).sum().item()
        
        # Validation phase
        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for data, target in val_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                loss = criterion(output, target)
                
                val_loss += loss.item()
                predicted = (output > 0.5).float()
                val_total += target.size(0)
                val_correct += (predicted == target).sum().item()
        
        # Calculate averages
        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)
        train_acc = 100 * train_correct / train_total
        val_acc = 100 * val_correct / val_total
        
        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)
        train_accuracies.append(train_acc)
        val_accuracies.append(val_acc)
        
        print(f'Epoch [{epoch+1}/{num_epochs}]')
        print(f'Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.2f}%')
        print(f'Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.2f}%')
        print('-' * 50)
    
    return train_losses, val_losses, train_accuracies, val_accuracies

# Train the PyTorch model
print("Training PyTorch LSTM Model...")
pytorch_train_losses, pytorch_val_losses, pytorch_train_accs, pytorch_val_accs = train_pytorch_model(
    pytorch_model, train_loader, val_loader, criterion, optimizer, num_epochs=25
)

In [ ]:
# Evaluation function for PyTorch
def evaluate_pytorch_model(model, test_loader):
    model.eval()
    predictions = []
    true_labels = []
    probabilities = []
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            probabilities.extend(output.cpu().numpy())
            predicted = (output > 0.5).float()
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(target.cpu().numpy())
    
    return np.array(true_labels), np.array(predictions), np.array(probabilities)

# Evaluate PyTorch model
print("Evaluating PyTorch LSTM Model...")
pytorch_y_true, pytorch_y_pred, pytorch_y_prob = evaluate_pytorch_model(pytorch_model, test_loader)

# Calculate metrics
pytorch_accuracy = accuracy_score(pytorch_y_true, pytorch_y_pred)
pytorch_precision = precision_score(pytorch_y_true, pytorch_y_pred)
pytorch_recall = recall_score(pytorch_y_true, pytorch_y_pred)
pytorch_f1 = f1_score(pytorch_y_true, pytorch_y_pred)
pytorch_auc = roc_auc_score(pytorch_y_true, pytorch_y_prob)

print("PyTorch LSTM Model Results:")
print(f"Accuracy: {pytorch_accuracy:.4f} ({pytorch_accuracy*100:.2f}%)")
print(f"Precision: {pytorch_precision:.4f}")
print(f"Recall: {pytorch_recall:.4f}")
print(f"F1-Score: {pytorch_f1:.4f}")
print(f"AUC: {pytorch_auc:.4f}")

# Classification report
print("\nClassification Report:")
print(classification_report(pytorch_y_true, pytorch_y_pred, target_names=['Non-Sarcastic', 'Sarcastic']))

## TensorFlow LSTM Model

In [ ]:
# TensorFlow data preprocessing
max_features = 10000  # Maximum number of words to keep
max_length = 50       # Maximum sequence length

# Create tokenizer
tokenizer = Tokenizer(num_words=max_features, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

# Convert texts to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post', truncating='post')
X_val_pad = pad_sequences(X_val_seq, maxlen=max_length, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post', truncating='post')

# Convert labels to numpy arrays
y_train_tf = np.array(y_train)
y_val_tf = np.array(y_val)
y_test_tf = np.array(y_test)

print(f"TensorFlow data shapes:")
print(f"X_train: {X_train_pad.shape}")
print(f"X_val: {X_val_pad.shape}")
print(f"X_test: {X_test_pad.shape}")
print(f"y_train: {y_train_tf.shape}")
print(f"y_val: {y_val_tf.shape}")
print(f"y_test: {y_test_tf.shape}")

# Vocabulary size
vocab_size_tf = min(max_features, len(tokenizer.word_index) + 1)
print(f"TensorFlow vocabulary size: {vocab_size_tf}")

In [ ]:
# TensorFlow LSTM Model
def create_lstm_model(vocab_size, embedding_dim=100, lstm_units=128, dropout=0.3):
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=max_length),
        Bidirectional(LSTM(lstm_units, return_sequences=True, dropout=dropout, recurrent_dropout=dropout)),
        Bidirectional(LSTM(lstm_units//2, dropout=dropout, recurrent_dropout=dropout)),
        Dense(64, activation='relu'),
        Dropout(dropout),
        Dense(32, activation='relu'),
        Dropout(dropout),
        Dense(1, activation='sigmoid')
    ])
    
    return model

# Create and compile the model
tensorflow_model = create_lstm_model(vocab_size_tf)
tensorflow_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("TensorFlow Model Summary:")
tensorflow_model.summary()

# Define callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=0.0001,
    verbose=1
)

# Train the model
print("Training TensorFlow LSTM Model...")
tensorflow_history = tensorflow_model.fit(
    X_train_pad, y_train_tf,
    batch_size=64,
    epochs=25,
    validation_data=(X_val_pad, y_val_tf),
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

In [ ]:
# Evaluate TensorFlow model
print("Evaluating TensorFlow LSTM Model...")

# Get predictions
tensorflow_y_prob = tensorflow_model.predict(X_test_pad)
tensorflow_y_pred = (tensorflow_y_prob > 0.5).astype(int).flatten()
tensorflow_y_prob = tensorflow_y_prob.flatten()

# Calculate metrics
tensorflow_accuracy = accuracy_score(y_test_tf, tensorflow_y_pred)
tensorflow_precision = precision_score(y_test_tf, tensorflow_y_pred)
tensorflow_recall = recall_score(y_test_tf, tensorflow_y_pred)
tensorflow_f1 = f1_score(y_test_tf, tensorflow_y_pred)
tensorflow_auc = roc_auc_score(y_test_tf, tensorflow_y_prob)

print("TensorFlow LSTM Model Results:")
print(f"Accuracy: {tensorflow_accuracy:.4f} ({tensorflow_accuracy*100:.2f}%)")
print(f"Precision: {tensorflow_precision:.4f}")
print(f"Recall: {tensorflow_recall:.4f}")
print(f"F1-Score: {tensorflow_f1:.4f}")
print(f"AUC: {tensorflow_auc:.4f}")

# Classification report
print("\nClassification Report:")
print(classification_report(y_test_tf, tensorflow_y_pred, target_names=['Non-Sarcastic', 'Sarcastic']))

## Hyperparameter Tuning with Keras Tuner

In [ ]:
# Hyperparameter tuning with Keras Tuner
def build_tuner_model(hp):
    model = Sequential()
    
    # Embedding layer
    model.add(Embedding(
        vocab_size_tf, 
        hp.Int('embedding_dim', min_value=50, max_value=200, step=50),
        input_length=max_length
    ))
    
    # LSTM layers
    model.add(Bidirectional(LSTM(
        hp.Int('lstm_units_1', min_value=64, max_value=256, step=64),
        return_sequences=True,
        dropout=hp.Float('dropout_1', min_value=0.2, max_value=0.5, step=0.1),
        recurrent_dropout=hp.Float('recurrent_dropout_1', min_value=0.2, max_value=0.5, step=0.1)
    )))
    
    model.add(Bidirectional(LSTM(
        hp.Int('lstm_units_2', min_value=32, max_value=128, step=32),
        dropout=hp.Float('dropout_2', min_value=0.2, max_value=0.5, step=0.1),
        recurrent_dropout=hp.Float('recurrent_dropout_2', min_value=0.2, max_value=0.5, step=0.1)
    )))
    
    # Dense layers
    model.add(Dense(
        hp.Int('dense_units', min_value=32, max_value=128, step=32),
        activation='relu'
    ))
    model.add(Dropout(hp.Float('final_dropout', min_value=0.2, max_value=0.5, step=0.1)))
    
    # Output layer
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Create tuner
tuner = kt.RandomSearch(
    build_tuner_model,
    objective='val_accuracy',
    max_trials=10,  # Reduced for faster execution
    directory='keras_tuner',
    project_name='sarcasm_detection'
)

print("Tuner search space:")
tuner.search_space_summary()

# Perform hyperparameter search
print("Starting hyperparameter search...")
tuner.search(
    X_train_pad, y_train_tf,
    epochs=10,  # Reduced epochs for faster tuning
    validation_data=(X_val_pad, y_val_tf),
    batch_size=64,
    verbose=1
)

# Get best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("\nBest hyperparameters:")
print(f"Embedding dim: {best_hps.get('embedding_dim')}")
print(f"LSTM units 1: {best_hps.get('lstm_units_1')}")
print(f"LSTM units 2: {best_hps.get('lstm_units_2')}")
print(f"Dense units: {best_hps.get('dense_units')}")
print(f"Learning rate: {best_hps.get('learning_rate')}")
print(f"Dropout 1: {best_hps.get('dropout_1')}")
print(f"Dropout 2: {best_hps.get('dropout_2')}")
print(f"Final dropout: {best_hps.get('final_dropout')}")

In [ ]:
# Train the best model
print("Training the best model with optimal hyperparameters...")
best_model = tuner.hypermodel.build(best_hps)

# Train with more epochs
best_history = best_model.fit(
    X_train_pad, y_train_tf,
    batch_size=64,
    epochs=25,
    validation_data=(X_val_pad, y_val_tf),
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# Evaluate the best model
print("Evaluating the best tuned model...")
best_y_prob = best_model.predict(X_test_pad)
best_y_pred = (best_y_prob > 0.5).astype(int).flatten()
best_y_prob = best_y_prob.flatten()

# Calculate metrics for best model
best_accuracy = accuracy_score(y_test_tf, best_y_pred)
best_precision = precision_score(y_test_tf, best_y_pred)
best_recall = recall_score(y_test_tf, best_y_pred)
best_f1 = f1_score(y_test_tf, best_y_pred)
best_auc = roc_auc_score(y_test_tf, best_y_prob)

print("Best Tuned LSTM Model Results:")
print(f"Accuracy: {best_accuracy:.4f} ({best_accuracy*100:.2f}%)")
print(f"Precision: {best_precision:.4f}")
print(f"Recall: {best_recall:.4f}")
print(f"F1-Score: {best_f1:.4f}")
print(f"AUC: {best_auc:.4f}")

# Classification report
print("\nClassification Report:")
print(classification_report(y_test_tf, best_y_pred, target_names=['Non-Sarcastic', 'Sarcastic']))

## Model Visualization and Analysis

In [ ]:
# Plot training history
plt.figure(figsize=(20, 15))

# PyTorch training plots
plt.subplot(3, 3, 1)
plt.plot(pytorch_train_losses, label='Train Loss', marker='o')
plt.plot(pytorch_val_losses, label='Validation Loss', marker='s')
plt.title('PyTorch LSTM - Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(3, 3, 2)
plt.plot(pytorch_train_accs, label='Train Accuracy', marker='o')
plt.plot(pytorch_val_accs, label='Validation Accuracy', marker='s')
plt.title('PyTorch LSTM - Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.grid(True)

# TensorFlow training plots
plt.subplot(3, 3, 4)
plt.plot(tensorflow_history.history['loss'], label='Train Loss', marker='o')
plt.plot(tensorflow_history.history['val_loss'], label='Validation Loss', marker='s')
plt.title('TensorFlow LSTM - Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(3, 3, 5)
plt.plot(tensorflow_history.history['accuracy'], label='Train Accuracy', marker='o')
plt.plot(tensorflow_history.history['val_accuracy'], label='Validation Accuracy', marker='s')
plt.title('TensorFlow LSTM - Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Best model training plots
plt.subplot(3, 3, 7)
plt.plot(best_history.history['loss'], label='Train Loss', marker='o')
plt.plot(best_history.history['val_loss'], label='Validation Loss', marker='s')
plt.title('Best Tuned Model - Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(3, 3, 8)
plt.plot(best_history.history['accuracy'], label='Train Accuracy', marker='o')
plt.plot(best_history.history['val_accuracy'], label='Validation Accuracy', marker='s')
plt.title('Best Tuned Model - Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Model comparison
plt.subplot(3, 3, 3)
models = ['PyTorch\nLSTM', 'TensorFlow\nLSTM', 'Best Tuned\nModel']
accuracies = [pytorch_accuracy * 100, tensorflow_accuracy * 100, best_accuracy * 100]
bars = plt.bar(models, accuracies, color=['skyblue', 'lightgreen', 'orange'])
plt.title('Model Accuracy Comparison')
plt.ylabel('Accuracy (%)')
plt.ylim(0, 100)

# Add value labels on bars
for bar, accuracy in zip(bars, accuracies):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
             f'{accuracy:.2f}%', ha='center', va='bottom', fontweight='bold')

# Add horizontal line at 80%
plt.axhline(y=80, color='red', linestyle='--', alpha=0.7, label='80% Target')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(3, 3, 6)
f1_scores = [pytorch_f1, tensorflow_f1, best_f1]
bars = plt.bar(models, f1_scores, color=['skyblue', 'lightgreen', 'orange'])
plt.title('Model F1-Score Comparison')
plt.ylabel('F1-Score')
plt.ylim(0, 1)

# Add value labels on bars
for bar, f1 in zip(bars, f1_scores):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{f1:.3f}', ha='center', va='bottom', fontweight='bold')

plt.grid(True, alpha=0.3)

plt.subplot(3, 3, 9)
auc_scores = [pytorch_auc, tensorflow_auc, best_auc]
bars = plt.bar(models, auc_scores, color=['skyblue', 'lightgreen', 'orange'])
plt.title('Model AUC Comparison')
plt.ylabel('AUC Score')
plt.ylim(0, 1)

# Add value labels on bars
for bar, auc in zip(bars, auc_scores):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{auc:.3f}', ha='center', va='bottom', fontweight='bold')

plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Confusion Matrix and ROC Curves
plt.figure(figsize=(20, 12))

# Confusion matrices
models_data = [
    ('PyTorch LSTM', pytorch_y_true, pytorch_y_pred, pytorch_y_prob),
    ('TensorFlow LSTM', y_test_tf, tensorflow_y_pred, tensorflow_y_prob),
    ('Best Tuned Model', y_test_tf, best_y_pred, best_y_prob)
]

for i, (model_name, y_true, y_pred, y_prob) in enumerate(models_data):
    # Confusion Matrix
    plt.subplot(2, 3, i+1)
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Non-Sarcastic', 'Sarcastic'],
                yticklabels=['Non-Sarcastic', 'Sarcastic'])
    plt.title(f'{model_name} - Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')

# ROC Curves
plt.subplot(2, 3, 4)
for model_name, y_true, y_pred, y_prob in models_data:
    fpr, tpr, _ = roc_curve(y_true, y_prob)
    auc_score = roc_auc_score(y_true, y_prob)
    plt.plot(fpr, tpr, label=f'{model_name} (AUC = {auc_score:.3f})', linewidth=2)

plt.plot([0, 1], [0, 1], 'k--', alpha=0.5)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves Comparison')
plt.legend()
plt.grid(True, alpha=0.3)

# Precision-Recall curves
plt.subplot(2, 3, 5)
from sklearn.metrics import precision_recall_curve, average_precision_score

for model_name, y_true, y_pred, y_prob in models_data:
    precision, recall, _ = precision_recall_curve(y_true, y_prob)
    avg_precision = average_precision_score(y_true, y_prob)
    plt.plot(recall, precision, label=f'{model_name} (AP = {avg_precision:.3f})', linewidth=2)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curves')
plt.legend()
plt.grid(True, alpha=0.3)

# Metrics comparison table
plt.subplot(2, 3, 6)
plt.axis('off')

metrics_data = []
for model_name, y_true, y_pred, y_prob in models_data:
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_prob)
    
    metrics_data.append([model_name, f'{accuracy:.3f}', f'{precision:.3f}', 
                        f'{recall:.3f}', f'{f1:.3f}', f'{auc:.3f}'])

table = plt.table(cellText=metrics_data,
                 colLabels=['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC'],
                 cellLoc='center',
                 loc='center')
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 1.5)
plt.title('Model Performance Metrics', pad=20)

plt.tight_layout()
plt.show()

## Model Saving and Conclusion

In [ ]:
# Save models
# Save PyTorch model
torch.save({
    'model_state_dict': pytorch_model.state_dict(),
    'vocab_to_int': vocab_to_int,
    'model_params': {
        'vocab_size': vocab_size,
        'embedding_dim': embedding_dim,
        'hidden_dim': hidden_dim,
        'num_layers': num_layers,
        'dropout': dropout
    }
}, 'pytorch_lstm_sarcasm_model.pth')

# Save TensorFlow models
tensorflow_model.save('tensorflow_lstm_sarcasm_model.keras')
best_model.save('best_tuned_lstm_sarcasm_model.keras')

# Save tokenizer
import pickle
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

print("Models saved successfully!")
print("Files saved:")
print("- pytorch_lstm_sarcasm_model.pth")
print("- tensorflow_lstm_sarcasm_model.keras")
print("- best_tuned_lstm_sarcasm_model.keras")
print("- tokenizer.pkl")

In [ ]:
# Final Summary
print("="*80)
print("LSTM SARCASM DETECTION MODEL SUMMARY")
print("="*80)

print(f"\nDataset Information:")
print(f"- Total samples: {len(df):,}")
print(f"- Training samples: {len(X_train):,}")
print(f"- Validation samples: {len(X_val):,}")
print(f"- Test samples: {len(X_test):,}")
print(f"- Sarcasm ratio: {df['is_sarcastic'].mean():.2%}")

print(f"\nModel Performance Summary:")
print(f"{'Model':<20} {'Accuracy':<10} {'Precision':<10} {'Recall':<10} {'F1-Score':<10} {'AUC':<10}")
print("-" * 70)
print(f"{'PyTorch LSTM':<20} {pytorch_accuracy:.3f}     {pytorch_precision:.3f}      {pytorch_recall:.3f}    {pytorch_f1:.3f}      {pytorch_auc:.3f}")
print(f"{'TensorFlow LSTM':<20} {tensorflow_accuracy:.3f}     {tensorflow_precision:.3f}      {tensorflow_recall:.3f}    {tensorflow_f1:.3f}      {tensorflow_auc:.3f}")
print(f"{'Best Tuned Model':<20} {best_accuracy:.3f}     {best_precision:.3f}      {best_recall:.3f}    {best_f1:.3f}      {best_auc:.3f}")

print(f"\nKey Achievements:")
print(f"✓ Implemented LSTM models in both PyTorch and TensorFlow")
print(f"✓ Comprehensive evaluation metrics (Accuracy, Precision, Recall, F1-Score, AUC)")
print(f"✓ Visualized training progress and model comparisons")
print(f"✓ Performed hyperparameter tuning with Keras Tuner")

# Check if 80% accuracy target is met
models_above_80 = []
if pytorch_accuracy >= 0.8:
    models_above_80.append("PyTorch LSTM")
if tensorflow_accuracy >= 0.8:
    models_above_80.append("TensorFlow LSTM")
if best_accuracy >= 0.8:
    models_above_80.append("Best Tuned Model")

if models_above_80:
    print(f"✓ Achieved 80%+ accuracy target with: {', '.join(models_above_80)}")
else:
    print(f"⚠ None of the models achieved the 80% accuracy target")
    print(f"  Best accuracy: {max(pytorch_accuracy, tensorflow_accuracy, best_accuracy):.3f}")

print(f"\nBest Overall Model: ", end="")
best_overall_acc = max(pytorch_accuracy, tensorflow_accuracy, best_accuracy)
if best_overall_acc == pytorch_accuracy:
    print("PyTorch LSTM")
elif best_overall_acc == tensorflow_accuracy:
    print("TensorFlow LSTM")
else:
    print("Best Tuned Model")

print(f"\nRecommendations:")
print(f"1. Use Google Colab with T4 GPU or TPU for faster training")
print(f"2. Consider ensemble methods to improve performance")
print(f"3. Explore pre-trained embeddings (Word2Vec, GloVe, BERT)")
print(f"4. Try different text preprocessing techniques")
print(f"5. Experiment with different model architectures (GRU, Transformer)")

print("="*80)

## Sample Predictions

Let's test our best model with some sample headlines to see how it performs on individual examples.

In [ ]:
# Function to make predictions on new text
def predict_sarcasm(text, model, tokenizer, max_length=50):
    """Predict if a headline is sarcastic or not"""
    # Clean the text
    cleaned_text = clean_text(text)
    
    # Convert to sequence
    sequence = tokenizer.texts_to_sequences([cleaned_text])
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding='post', truncating='post')
    
    # Make prediction
    probability = model.predict(padded_sequence, verbose=0)[0][0]
    prediction = 1 if probability > 0.5 else 0
    
    return prediction, probability

# Test with sample headlines
sample_headlines = [
    "Breaking: Local man discovers amazing new way to waste time",
    "Scientists announce breakthrough in cancer research",
    "Area man's opinion on politics changes absolutely everything",
    "New study shows exercise may be good for your health",
    "Local woman shocked to discover water is wet",
    "Government announces new policy to help citizens",
    "Man wins lottery, quits job to pursue dream of being unemployed",
    "Research reveals shocking truth: people need food to survive"
]

print("Sample Sarcasm Predictions using Best Tuned Model:")
print("="*80)
print(f"{'Headline':<60} {'Predicted':<12} {'Probability':<12}")
print("-"*80)

for headline in sample_headlines:
    prediction, probability = predict_sarcasm(headline, best_model, tokenizer)
    prediction_text = "Sarcastic" if prediction == 1 else "Not Sarcastic"
    print(f"{headline[:58]:<60} {prediction_text:<12} {probability:.3f}")

print("="*80)

# Let's also show some actual examples from our test set with their predictions
print("\nActual Test Examples vs Predictions:")
print("="*80)
print(f"{'Headline':<50} {'Actual':<12} {'Predicted':<12} {'Probability':<12}")
print("-"*80)

# Show first 10 test examples
for i in range(min(10, len(X_test))):
    actual = y_test_tf[i]
    prediction, probability = predict_sarcasm(X_test[i], best_model, tokenizer)
    actual_text = "Sarcastic" if actual == 1 else "Not Sarcastic"
    prediction_text = "Sarcastic" if prediction == 1 else "Not Sarcastic"
    
    # Truncate headline for display
    headline_display = X_test[i][:48] + "..." if len(X_test[i]) > 48 else X_test[i]
    
    print(f"{headline_display:<50} {actual_text:<12} {prediction_text:<12} {probability:.3f}")

print("="*80)